In [1]:
import pandas as pd
import os
data_dirpath = '/usr2/mamille2/tumblr/data'

# See if can use data that already have for reblog prediction

## Load followers

In [2]:
follower_data = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(follower_data))
print(follower_data.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

## Load followees

In [2]:
follow_data = pd.read_pickle(os.path.join(data_dirpath, 'follow_data_recent100.pkl'))
print(len(follow_data))
print(follow_data.columns)

329702060
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


## Get followees (now a script)

In [11]:
# Test follow structure

follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, iterator=True)
follow_data = chunked.get_chunk(chunksize)
print(len(follow_data))
print(follow_data.columns)

100000
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


In [3]:
# See what follow structure can find for set of followers
from tqdm import tqdm_notebook as tqdm
import numpy as np

follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
total_iters = int(np.ceil(3220088178 / chunksize))
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, chunksize=chunksize)

stx_follower_ids = set()
# follower_ids = follower_data['tumblog_id'].tolist()
# followee_data = []

for chunk in tqdm(chunked, total=total_iters):
    stx_follower_ids |= set(chunk['tumblog_id'].unique())
#     followee_data.append(chunk[chunk['tumblog_id'].isin(follower_ids)].values)
    
    
print(len(stx_follower_ids))

follower_ids = stx_follower_ids.intersection(set(follower_data['tumblog_id']))
len(follower_ids) # Have at least one piece of follow information for these users


100861970


3044188

In [5]:
# Save out follower ids
with open(os.path.join(data_dirpath, 'halfday_followers_with_descriptions.txt'), 'w') as f:
    for i in sorted(follower_ids):
        f.write(f'{i}\n')

In [4]:
# Get followee data
follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
total_iters = int(np.ceil(3220088178 / chunksize))
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, chunksize=chunksize)

followee_data = [] # list of np arrays to be concatenated

for chunk in tqdm(chunked, total=total_iters):
    followee_data.append(chunk[chunk['tumblog_id'].isin(follower_ids)].values)
    
follower_follow_data = pd.DataFrame(np.vstack(followee_data), columns=hdr)
len(follower_follow_data)

KeyboardInterrupt: 

In [29]:
# Save follow data from followers for which have descriptions
out_fname = 'follow_data_recent100.pkl'

follower_follow_data.to_pickle(os.path.join(data_dirpath, out_fname))

## Get followee descriptions

In [3]:
# Load blog descriptions
descs = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(descs))
print(descs.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

In [5]:
followees = set(follow_data['followed_tumblog_id'])
print(len(followees))

followees_with_descriptions = followees.intersection(set(descs['tumblog_id']))
print(len(followees_with_descriptions))

11816219
1454160


##  Select posts from followees, posted after followers followed them

In [6]:
# Load post data

post_fname = 'textposts_recent100.pkl' # ~75 GB in memory
post_data = pd.read_pickle(os.path.join(data_dirpath, post_fname))
print(len(post_data))
print(post_data.columns)

27342192
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


In [8]:
followee_posts = post_data[post_data['tumblog_id'].isin(followees_with_descriptions)] # all posts regardless of time posted
print(len(followee_posts))

14852224


In [9]:
# Save followee posts
followee_posts.to_pickle(os.path.join(data_dirpath, 'followee_posts_recent100.pkl'))

In [2]:
# Load followee posts
followee_posts = pd.read_pickle(os.path.join(data_dirpath, 'followee_posts_recent100.pkl'))
print(len(followee_posts))
print(followee_posts.columns)

14852224
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


### Merge text and user info for quick prediction (duplicate posts for followees with multiple followers in the dataset)
Would filter followee posts based on timestamp of follows here

#### Make a dictionary of followees -> followers

In [3]:
follow_data = pd.read_pickle(os.path.join(data_dirpath, 'follow_data_recent100.pkl'))
print(len(follow_data))
print(follow_data.columns)

329702060
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


In [5]:
gped = follow_data.set_index('tumblog_id').groupby('followed_tumblog_id')
gped.groups

{1: Int64Index([ 81825698, 208219453, 233717549, 295109096, 302317237, 163655147,
             232576315, 250679697, 186469847, 220082623,
             ...
              48357585,  76493669, 141966943, 192114312, 232273459, 265408623,
             267930039, 293204551, 315104279, 324467431],
            dtype='int64', name='tumblog_id', length=6287),
 2: Int64Index([    31731,   1586104,   4458421, 295124622, 255298629,   4675291,
               3171583, 301084370,    518805,   2908047, 337653955,   2104791,
               3635302,  13674588,  10746550, 228983798,   2419661,  12882499,
                647104,   3190058, 260862065,  14032575,   6222664,   3250258,
               6168064,     74743, 253283632,   8531702,      6346,   8867780,
             303161996,  69697753,  10035498,  10785644,       438,   5897976,
               8516343, 251377050,  12554163,   6911517,  10899465, 177427246,
              25872967,   2299424,   2710338,     11307, 250342089,    268554,
            

In [6]:
len(gped.groups)

11816219

In [8]:
gped.groups[1]

Int64Index([ 81825698, 208219453, 233717549, 295109096, 302317237, 163655147,
            232576315, 250679697, 186469847, 220082623,
            ...
             48357585,  76493669, 141966943, 192114312, 232273459, 265408623,
            267930039, 293204551, 315104279, 324467431],
           dtype='int64', name='tumblog_id', length=6287)

In [12]:
follow_dict = {key: list(gped.groups[key]) for key in gped.groups}
print(len(follow_dict))
follow_dict[1]

11816219


[81825698,
 208219453,
 233717549,
 295109096,
 302317237,
 163655147,
 232576315,
 250679697,
 186469847,
 220082623,
 231629031,
 235842979,
 244430730,
 39785959,
 112789979,
 261568963,
 297115643,
 40071418,
 216456988,
 287686260,
 291961804,
 308897780,
 321936036,
 85897188,
 164185201,
 219970969,
 223295564,
 225882705,
 227967488,
 227977586,
 228580569,
 233094065,
 239910153,
 253441869,
 269850717,
 272008897,
 317550237,
 319259349,
 2302847,
 60013017,
 78178668,
 230722414,
 240808499,
 245080393,
 281175694,
 289494206,
 297362030,
 334476334,
 1753021,
 4020574,
 79498292,
 111296988,
 161830481,
 174866811,
 231347604,
 237490708,
 238117575,
 248707999,
 298342975,
 299797767,
 317334663,
 155766778,
 185750849,
 233558643,
 238113092,
 310333568,
 312232120,
 325930776,
 246374331,
 246703916,
 265568593,
 293035009,
 328820345,
 93209678,
 201450194,
 234092799,
 240340498,
 247851922,
 265366170,
 305394794,
 317992906,
 322202922,
 335444322,
 97576862,
 221320

In [25]:
# Average #followers/follower
len([d for l in follow_dict.values() for d in l]) / len(follow_dict)

27.90250079149684

In [15]:
# Save
import pickle

with open(os.path.join(data_dirpath, 'follow_recent100_dict.pkl'), 'wb') as f:
    pickle.dump(follow_dict, f)

In [19]:
# Sample from posts

posts = followee_posts.sample(int(1e4))
print(len(posts))
posts.columns

10000

In [28]:
posts.columns

Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')

In [36]:
# Merge posts with follower/followee names, sample max 10 followers
import random

def safe_sample(l, t):
    if len(l) > t:
        return random.sample(l, t)
    else:
        return l

posts['followers'] = posts['tumblog_id'].map(lambda x: safe_sample(follow_dict[x], 10))

In [44]:
s = posts.apply(lambda x: pd.Series(x['followers']), axis=1).stack().reset_index(level=1, drop=True)
print(len(s))

54447


AttributeError: 'Series' object has no attribute 'columns'

In [46]:
s.name = 'follower'
data = posts.join(s)
print(len(data))

54447
          post_id activity_time_epoch  tumblog_id  \
603  157090203909       1486791467000   257811292   
603  157090203909       1486791467000   257811292   
603  157090203909       1486791467000   257811292   
603  157090203909       1486791467000   257811292   
603  157090203909       1486791467000   257811292   

                                  post_title                  post_short_url  \
603  trying to get your story together like:  https://tmblr.co/ZWtTEj2IJJaa5   
603  trying to get your story together like:  https://tmblr.co/ZWtTEj2IJJaa5   
603  trying to get your story together like:  https://tmblr.co/ZWtTEj2IJJaa5   
603  trying to get your story together like:  https://tmblr.co/ZWtTEj2IJJaa5   
603  trying to get your story together like:  https://tmblr.co/ZWtTEj2IJJaa5   

    post_type post_caption post_format post_note_count created_time_epoch  \
603      text          NaN    markdown           45194      1486791467000   
603      text          NaN    markdown  

In [48]:
data.reset_index(drop=True, inplace=True)

In [52]:
data.drop('followers', axis=1, inplace=True)

#### Merge in follower, followee blog description info

In [49]:
# Load blog descriptions
descs = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(descs))
print(descs.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

In [54]:
# Merge in followee info
merged = pd.merge(data, descs, on='tumblog_id', how='left', suffixes=('_post', '_desc'))
print(len(merged))
print(merged.columns)

54447
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch_post', 'updated_time_epoch_post',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier_post', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1', 'follower',
       'activity_time_epoch_desc', 'tumblr_blog_name', 'tumblr_blog_title',
       'tumblr_blog_description', 'tumblr_blog_url', 'tumblr_blog_theme',
       'is_group_blog', 'is_primary', 'is_private', 'created_time_epoch_desc',
       'updated_time_epoch_desc', 'timezone', 'language',
       'blog_classifier_desc', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',

In [55]:
# Merge in follower info
merged = pd.merge(merged, descs, left_on='follower', right_on='tumblog_id', how='left', suffixes=('_followee', '_follower'))
print(len(merged))
print(merged.columns)

54447
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id_followee',
       'post_title', 'post_short_url', 'post_type', 'post_caption',
       'post_format', 'post_note_count', 'created_time_epoch_post',
       ...
       'ethnicity/nationality_hegemonic_present_follower',
       'gender_hegemonic_present_follower',
       'pronouns_hegemonic_present_follower',
       'sexual orientation_hegemonic_present_follower',
       'ethnicity/nationality_opposite_present_follower',
       'gender_opposite_present_follower',
       'pronouns_opposite_present_follower',
       'sexual orientation_opposite_present_follower',
       'gender/sexuality_opposite_present_follower',
       'gender/sexuality_hegemonic_present_follower'],
      dtype='object', length=134)


In [56]:
merged['parsed_blog_description_followee']

0        boy I can't wait til i actually do stuff with ...
1        boy I can't wait til i actually do stuff with ...
2        boy I can't wait til i actually do stuff with ...
3        boy I can't wait til i actually do stuff with ...
4        boy I can't wait til i actually do stuff with ...
5        boy I can't wait til i actually do stuff with ...
6        boy I can't wait til i actually do stuff with ...
7        boy I can't wait til i actually do stuff with ...
8        boy I can't wait til i actually do stuff with ...
9        boy I can't wait til i actually do stuff with ...
10       Welcome one and all to my small corner of Tumb...
11       Welcome one and all to my small corner of Tumb...
12       Welcome one and all to my small corner of Tumb...
13       Jak / 22 / taurus FE / ygo / mankin / tales of...
14       Jak / 22 / taurus FE / ygo / mankin / tales of...
15       Jak / 22 / taurus FE / ygo / mankin / tales of...
16       Jak / 22 / taurus FE / ygo / mankin / tales of.

In [57]:
merged['parsed_blog_description_follower']

0                                     Sunrises and sunsets
1                          The blog of Green Giant Uranus.
2        Dove | Christian | Astronomer | Artist | ENTJ ...
3        Hi I'm Rain and I'm the Bird Princess icon by ...
4        Just my art blog: @sleepistoxic -- Im a young,...
5        Wtf am I doing| She/Her| 22| Icon credit @moos...
6                                 17/cartoons/bands/ghosts
7        ashamed owner of @reallybadcookerunshitposts (...
8        Hi! I'm Liz. I draw things and have a lot of f...
9        infp w an emphatic i // ravenclaw // leo // po...
10       Just random stuff that I enjoy from fandoms I ...
11       Hey hows it going. I'm here to meme and shit w...
12                       Anything that I feel like posting
13       31 / ♂ / American / ENTP / Dinosaur. No relati...
14       On hiatus till I finish my projects. Header an...
15                                          gemini pierced
16                                              Praise K

In [58]:
# Save out merged doc
merged.to_pickle(os.path.join(data_dirpath, 'posts_descs_54k.pkl'))

## Get outcome measure: whether or not a post was reblogged

In [2]:
# Load post data

post_fname = 'textposts_recent100.pkl' # ~75 GB in memory
post_data = pd.read_pickle(os.path.join(data_dirpath, post_fname))
print(len(post_data))
print(post_data.columns)

27342192
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


In [3]:
# Load data to have outcome added to
data = pd.read_pickle(os.path.join(data_dirpath, 'posts_descs_54k.pkl'))
print(len(data))
print(data.columns)

54447
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id_followee',
       'post_title', 'post_short_url', 'post_type', 'post_caption',
       'post_format', 'post_note_count', 'created_time_epoch_post',
       ...
       'ethnicity/nationality_hegemonic_present_follower',
       'gender_hegemonic_present_follower',
       'pronouns_hegemonic_present_follower',
       'sexual orientation_hegemonic_present_follower',
       'ethnicity/nationality_opposite_present_follower',
       'gender_opposite_present_follower',
       'pronouns_opposite_present_follower',
       'sexual orientation_opposite_present_follower',
       'gender/sexuality_opposite_present_follower',
       'gender/sexuality_hegemonic_present_follower'],
      dtype='object', length=134)


In [12]:
# Any reblogs?
from tqdm import tqdm_notebook as tqdm

# def is_reblog(x):
#     rows = post_data[(post_data['reblogged_from_post_id'] == x['post_id']) & \
#                                   (post_data['tumblog_id'] == x['tumblog_id_follower'])]
# data['reblogged'] = data.apply(is_reblog, axis=1)

def is_reblog(post_id, tumblog_id_follower):
    
    if not post_id in post_data['reblogged_from_post_id']:
        return False
    
    else:
        rows = post_data[(post_data['reblogged_from_post_id'] == post_id) & \
                                      (post_data['tumblog_id'] == tumblog_id_follower)]
        if len(rows) > 1:
            return True
        else:
            return False

data['reblogged'] = [is_reblog(post_id, tumblog_id_follower) for post_id, tumblog_id_follower \
                     in tqdm(zip(data['post_id'], data['tumblog_id_follower']), total=len(data))]

sum(data['reblogged'])

0

In [5]:
print(data['reblogged_from_post_id'].dtype)
print(data['post_id'].dtype)

object
object


In [6]:
data['reblogged_from_post_id'].head()

0    1.5709e+11
1    1.5709e+11
2    1.5709e+11
3    1.5709e+11
4    1.5709e+11
Name: reblogged_from_post_id, dtype: object

In [10]:
reblogged = data['reblogged_from_post_id'].fillna(-1).astype(int)
reblogged.dtype

dtype('int64')

In [11]:
# reblogged_posts = post_data['reblogged_from_post_id'].unique()
reblogged_posts = reblogged.unique()
len(reblogged_posts)

7920

In [13]:
posts = data['post_id'].astype(int)
posts.dtype

dtype('int64')

In [14]:
intersect = set(posts).intersection(set(reblogged))
len(intersect)

2